In [5]:
import pandas as pd
import numpy as np
import json
import re
from typing import List, Dict, Tuple, Set, Optional, Any
import networkx as nx
from collections import defaultdict, deque
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error, brier_score_loss
from scipy import stats
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings('ignore')

class SprintPlanner:
    def __init__(self, 
                 team_id: int = 1,
                 theta: float = 0.8,  # Meta de confiabilidade
                 n_simulations: int = 5000,
                 lambda_spillover: float = 0.1,  # Penalização por spillover
                 random_state: int = 42):
        """
        Sistema de planejamento de sprint com otimização probabilística
        
        Args:
            team_id: ID do time para filtrar dados históricos
            theta: Meta de confiabilidade (P(on-time) >= theta)
            n_simulations: Número de simulações Monte Carlo
            lambda_spillover: Fator de penalização por spillover
            random_state: Seed para reprodutibilidade
        """
        self.team_id = team_id
        self.theta = theta
        self.n_simulations = n_simulations
        self.lambda_spillover = lambda_spillover
        self.random_state = random_state
        np.random.seed(random_state)
        
        # Modelos preditivos
        self.capacity_model = None
        self.overrun_model = None
        self.spillover_model = None
        
        # Dados processados
        self.backlog_df = None
        self.dependency_graph = None
        self.capacity_distribution = None
        self.overrun_distributions = {}
        self.spillover_probabilities = {}
        
    def load_and_process_data(self, 
                             trilha_b1_path: str,
                             trilha_b2_path: str, 
                             trilha_b3_path: str,
                             trilha_b4_path: str,
                             backlog_priorizado_path: str) -> None:
        """Carrega e processa todos os arquivos de entrada"""
        
        print("1. Carregando arquivos...")
        
        # Carregar dados
        self.b1_df = pd.read_csv(trilha_b1_path)  # Capacidade histórica
        self.b2_df = pd.read_csv(trilha_b2_path)  # Histórico por item
        self.b3_df = pd.read_csv(trilha_b3_path)  # Capacidade por dev
        self.b4_df = pd.read_csv(trilha_b4_path)  # Backlog atual
        self.priorizado_df = pd.read_csv(backlog_priorizado_path)  # Saída Trilha A
        
        # Normalizar colunas
        self._normalize_columns()
        
        print("2. Preparando dados...")
        
        # Preparar backlog principal
        self._prepare_backlog()
        
        # Construir grafo de dependências
        self._build_dependency_graph()
        
        print("3. Treinando modelos preditivos...")
        
        # Treinar modelos
        self._train_capacity_model()
        self._train_overrun_model()
        self._train_spillover_model()
        
        print("✅ Dados carregados e modelos treinados!")
    
    def _normalize_columns(self) -> None:
        """Normaliza nomes das colunas"""
        # Corrigir coluna problemática em B3
        if 'horas disponiveis ' in self.b3_df.columns:
            self.b3_df = self.b3_df.rename(columns={'horas disponiveis ': 'horas_disponiveis'})
        
        # Padronizar nomes
        for df in [self.b1_df, self.b2_df, self.b3_df, self.b4_df, self.priorizado_df]:
            df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
    
    def _prepare_backlog(self) -> None:
        """Prepara o backlog principal combinando dados das trilhas"""
        # Começar com B4 (backlog atual)
        self.backlog_df = self.b4_df.copy()
        
        # Garantir que issueid seja string para joins
        self.backlog_df['issueid'] = self.backlog_df['issueid'].astype(str)
        self.priorizado_df['id'] = self.priorizado_df['id'].astype(str)
        
        # Juntar com dados de valor (Trilha A)
        valor_cols = ['id', 'roi_ajustado', 'score_ml', 'dep_impact']
        if all(col in self.priorizado_df.columns for col in valor_cols):
            valor_df = self.priorizado_df[valor_cols].copy()
            self.backlog_df = self.backlog_df.merge(
                valor_df, 
                left_on='issueid', 
                right_on='id', 
                how='left'
            )
            # Usar score_ml calibrado como valor principal
            self.backlog_df['valor_trilha_a'] = self.backlog_df['score_ml'].fillna(
                self.backlog_df['roi_ajustado'].fillna(
                    self.backlog_df['prioridade_score']
                )
            )
        else:
            # Fallback: usar prioridade_score
            self.backlog_df['valor_trilha_a'] = self.backlog_df['prioridade_score']
        
        # Parsear dependências
        self.backlog_df['deps_parsed'] = self.backlog_df['dependencias'].apply(self._parse_dependencies)
        
        print(f"   Backlog preparado: {len(self.backlog_df)} itens")
    
    def _parse_dependencies(self, deps_str: Any) -> List[str]:
        """Parseia string de dependências em lista"""
        if pd.isna(deps_str) or deps_str == '':
            return []
        
        deps_str = str(deps_str).strip()
        if not deps_str:
            return []
        
        # Tentar parsear como JSON
        try:
            if deps_str.startswith('[') and deps_str.endswith(']'):
                return json.loads(deps_str)
        except:
            pass
        
        # Parsear com separadores
        deps = []
        for sep in [',', ';']:
            if sep in deps_str:
                deps = [dep.strip().strip('"\'') for dep in deps_str.split(sep)]
                break
        else:
            deps = [deps_str.strip().strip('"\'')]
        
        return [str(dep) for dep in deps if dep]
    
    def _build_dependency_graph(self) -> None:
        """Constrói grafo de dependências"""
        self.dependency_graph = nx.DiGraph()
        
        # Todos os IDs válidos
        valid_ids = set(self.backlog_df['issueid'].astype(str))
        self.dependency_graph.add_nodes_from(valid_ids)
        
        # Adicionar arestas e detectar dependências externas
        external_deps = defaultdict(list)
        
        for _, row in self.backlog_df.iterrows():
            item_id = str(row['issueid'])
            
            for dep in row['deps_parsed']:
                dep = str(dep)
                if dep in valid_ids:
                    # Aresta: dependência → item
                    self.dependency_graph.add_edge(dep, item_id)
                else:
                    external_deps[item_id].append(dep)
        
        # Marcar dependências externas
        self.backlog_df['has_external_deps'] = self.backlog_df['issueid'].astype(str).map(
            lambda x: len(external_deps[x]) > 0
        )
        
        # Detectar e quebrar ciclos se necessário
        if not nx.is_directed_acyclic_graph(self.dependency_graph):
            self._break_cycles()
        
        print(f"   Grafo: {len(self.dependency_graph.nodes())} nós, {len(self.dependency_graph.edges())} arestas")
    
    def _break_cycles(self) -> None:
        """Quebra ciclos no grafo"""
        valor_dict = dict(zip(
            self.backlog_df['issueid'].astype(str), 
            self.backlog_df['valor_trilha_a']
        ))
        
        while not nx.is_directed_acyclic_graph(self.dependency_graph):
            try:
                cycle = nx.find_cycle(self.dependency_graph)
                min_weight = float('inf')
                edge_to_remove = None
                
                for u, v in cycle:
                    weight = valor_dict.get(u, 0) + valor_dict.get(v, 0)
                    if weight < min_weight:
                        min_weight = weight
                        edge_to_remove = (u, v)
                
                if edge_to_remove:
                    self.dependency_graph.remove_edge(*edge_to_remove)
                    print(f"   Ciclo quebrado: removida aresta {edge_to_remove}")
                else:
                    break
            except nx.NetworkXNoCycle:
                break
    
    def _train_capacity_model(self) -> None:
        """Treina modelo de previsão de capacidade"""
        # Filtrar por team
        team_data = self.b1_df[self.b1_df['teamid'] == self.team_id].copy()
        
        if len(team_data) < 3:
            print(f"   Poucos dados para team {self.team_id}, usando média histórica")
            mean_capacity = team_data['sp_entregues'].mean() if len(team_data) > 0 else 10
            std_capacity = team_data['sp_entregues'].std() if len(team_data) > 1 else 2
            self.capacity_distribution = {
                'mean': mean_capacity,
                'std': max(std_capacity, 1.0),
                'type': 'normal'
            }
            return
        
        # Ordenar por sprint
        team_data = team_data.sort_values('sprintid')
        ts = team_data['sp_entregues'].values
        
        try:
            # Tentar ETS
            model = ExponentialSmoothing(ts, trend='add', seasonal=None)
            fitted_model = model.fit()
            
            # Prever próximo valor
            forecast = fitted_model.forecast(steps=1)
            residuals = fitted_model.resid
            
            self.capacity_distribution = {
                'mean': float(forecast[0]),
                'std': float(np.std(residuals)) if len(residuals) > 1 else 2.0,
                'type': 'normal'
            }
            
        except Exception as e:
            # Fallback: usar média e desvio históricos
            self.capacity_distribution = {
                'mean': float(np.mean(ts)),
                'std': float(np.std(ts)) if len(ts) > 1 else 2.0,
                'type': 'normal'
            }
        
        print(f"   Capacidade: μ={self.capacity_distribution['mean']:.1f}, σ={self.capacity_distribution['std']:.1f}")
    
    def _train_overrun_model(self) -> None:
        """Treina modelo de overrun (razão realizado/estimado)"""
        # Preparar dados de overrun
        self.b2_df['overrun_ratio'] = self.b2_df['storypoints_realizados'] / self.b2_df['storypoints_estimados']
        self.b2_df['log_overrun'] = np.log(self.b2_df['overrun_ratio'])
        
        # Features disponíveis
        features = []
        X_data = pd.DataFrame()
        
        # SP estimados (binned)
        X_data['sp_estimados'] = self.b2_df['storypoints_estimados']
        X_data['sp_bin'] = pd.cut(X_data['sp_estimados'], bins=[0, 2, 5, 8, float('inf')], labels=[0, 1, 2, 3])
        features.extend(['sp_estimados', 'sp_bin'])
        
        # Spillover como feature
        if 'spillover_(dep_externa)' in self.b2_df.columns:
            X_data['spillover'] = self.b2_df['spillover_(dep_externa)']
            features.append('spillover')
        
        # Alvo
        y = self.b2_df['log_overrun'].fillna(0)
        
        if len(X_data) > 10:
            try:
                # Treinar LightGBM
                self.overrun_model = lgb.LGBMRegressor(
                    n_estimators=100,
                    random_state=self.random_state,
                    verbose=-1
                )
                self.overrun_model.fit(X_data[features], y)
                
                # Calcular resíduos para incerteza
                predictions = self.overrun_model.predict(X_data[features])
                residuals = y - predictions
                residual_std = np.std(residuals)
                
                print(f"   Modelo overrun treinado (MAE: {mean_absolute_error(y, predictions):.3f})")
                
            except Exception as e:
                print(f"   Erro no modelo overrun, usando médias: {e}")
                self.overrun_model = None
                residual_std = np.std(y)
        else:
            self.overrun_model = None
            residual_std = np.std(y) if len(y) > 1 else 0.2
        
        # Distribuições por faixas de SP
        for sp_range in [(0, 2), (2, 5), (5, 8), (8, float('inf'))]:
            mask = (self.b2_df['storypoints_estimados'] >= sp_range[0]) & \
                   (self.b2_df['storypoints_estimados'] < sp_range[1])
            
            if mask.sum() > 0:
                range_data = self.b2_df[mask]['log_overrun']
                self.overrun_distributions[sp_range] = {
                    'mean': float(range_data.mean()),
                    'std': max(float(range_data.std()), residual_std, 0.1)
                }
            else:
                self.overrun_distributions[sp_range] = {
                    'mean': 0.0,
                    'std': residual_std
                }
    
    def _train_spillover_model(self) -> None:
        """Treina modelo de probabilidade de spillover"""
        if 'spillover_(dep_externa)' not in self.b2_df.columns:
            # Criar probabilidades base por faixa de SP
            for sp_range in [(0, 2), (2, 5), (5, 8), (8, float('inf'))]:
                self.spillover_probabilities[sp_range] = 0.1  # 10% base
            return
        
        # Agrupar por faixas de SP e presença de dep externa
        self.b2_df['sp_bin'] = pd.cut(
            self.b2_df['storypoints_estimados'], 
            bins=[0, 2, 5, 8, float('inf')],
            labels=['small', 'medium', 'large', 'xlarge']
        )
        
        # Calcular probabilidades por faixa
        spillover_rates = self.b2_df.groupby('sp_bin')['spillover_(dep_externa)'].mean()
        
        for i, sp_range in enumerate([(0, 2), (2, 5), (5, 8), (8, float('inf'))]):
            bin_label = ['small', 'medium', 'large', 'xlarge'][i]
            if bin_label in spillover_rates.index:
                self.spillover_probabilities[sp_range] = max(0.05, min(0.5, spillover_rates[bin_label]))
            else:
                self.spillover_probabilities[sp_range] = 0.1
        
        print(f"   Spillover rates: {dict(zip(['S', 'M', 'L', 'XL'], self.spillover_probabilities.values()))}")
    
    def _get_sp_range(self, sp: float) -> Tuple[float, float]:
        """Retorna a faixa de SP para um valor"""
        for sp_range in [(0, 2), (2, 5), (5, 8), (8, float('inf'))]:
            if sp >= sp_range[0] and sp < sp_range[1]:
                return sp_range
        return (8, float('inf'))
    
    def monte_carlo_simulation(self) -> Dict[str, np.ndarray]:
        """Executa simulação Monte Carlo"""
        print(f"4. Executando {self.n_simulations} simulações Monte Carlo...")
        
        results = {
            'capacity_samples': np.zeros(self.n_simulations),
            'overrun_samples': {},  # Por item
            'sp_efetivo_samples': {}  # Por item
        }
        
        # Amostrar capacidades
        results['capacity_samples'] = np.random.normal(
            self.capacity_distribution['mean'],
            self.capacity_distribution['std'],
            self.n_simulations
        )
        results['capacity_samples'] = np.maximum(results['capacity_samples'], 1)  # Mínimo 1 SP
        
        # Amostrar overrun por item
        for _, item in self.backlog_df.iterrows():
            item_id = str(item['issueid'])
            sp = item['storypoints']
            sp_range = self._get_sp_range(sp)
            
            if self.overrun_model is not None:
                # Usar modelo se disponível
                try:
                    item_features = pd.DataFrame({
                        'sp_estimados': [sp],
                        'sp_bin': [self._get_sp_bin(sp)],
                        'spillover': [int(item.get('has_external_deps', False))]
                    })
                    
                    log_overrun_pred = self.overrun_model.predict(item_features)[0]
                    log_overrun_std = self.overrun_distributions[sp_range]['std']
                    
                    log_overrun_samples = np.random.normal(
                        log_overrun_pred, 
                        log_overrun_std, 
                        self.n_simulations
                    )
                except:
                    # Fallback para distribuição por faixa
                    dist = self.overrun_distributions[sp_range]
                    log_overrun_samples = np.random.normal(
                        dist['mean'], 
                        dist['std'], 
                        self.n_simulations
                    )
            else:
                # Usar distribuição por faixa de SP
                dist = self.overrun_distributions[sp_range]
                log_overrun_samples = np.random.normal(
                    dist['mean'], 
                    dist['std'], 
                    self.n_simulations
                )
            
            # Converter para overrun ratio
            overrun_samples = np.exp(log_overrun_samples)
            overrun_samples = np.maximum(overrun_samples, 0.5)  # Mínimo 50% do estimado
            overrun_samples = np.minimum(overrun_samples, 3.0)  # Máximo 300% do estimado
            
            results['overrun_samples'][item_id] = overrun_samples
            results['sp_efetivo_samples'][item_id] = sp * overrun_samples
        
        return results
    
    def _get_sp_bin(self, sp: float) -> int:
        """Converte SP em bin categórico"""
        if sp <= 2:
            return 0
        elif sp <= 5:
            return 1
        elif sp <= 8:
            return 2
        else:
            return 3
    
    def optimize_sprint(self, simulation_results: Dict) -> Tuple[List[str], Dict]:
        """Otimiza seleção de itens para o sprint"""
        print("5. Otimizando seleção de itens...")
        
        # Algoritmo guloso com chance constraints
        best_sprint = []
        best_value = 0
        best_prob_on_time = 0
        
        # Criar lista de candidatos ordenados por ganho marginal esperado
        candidates = []
        
        for _, item in self.backlog_df.iterrows():
            item_id = str(item['issueid'])
            sp_range = self._get_sp_range(item['storypoints'])
            spillover_prob = self.spillover_probabilities.get(sp_range, 0.1)
            
            # Penalização por spillover
            spillover_penalty = 1.0 - self.lambda_spillover * spillover_prob
            if item.get('has_external_deps', False):
                spillover_penalty *= 0.8  # Penalização adicional por dep externa
            
            # Ganho marginal esperado
            expected_sp_efetivo = np.mean(simulation_results['sp_efetivo_samples'][item_id])
            marginal_gain = (item['valor_trilha_a'] * spillover_penalty) / expected_sp_efetivo
            
            candidates.append({
                'id': item_id,
                'value': item['valor_trilha_a'],
                'marginal_gain': marginal_gain,
                'sp_estimado': item['storypoints'],
                'deps': item['deps_parsed']
            })
        
        # Ordenar por ganho marginal
        candidates.sort(key=lambda x: x['marginal_gain'], reverse=True)
        
        # Seleção gulosa respeitando dependências
        current_sprint = self._greedy_selection_with_dependencies(
            candidates, simulation_results
        )
        
        # Calcular probabilidade de sucesso
        prob_on_time = self._calculate_success_probability(current_sprint, simulation_results)
        
        # Ajustar para atingir meta theta
        if prob_on_time < self.theta:
            current_sprint = self._adjust_for_reliability(
                current_sprint, simulation_results
            )
            prob_on_time = self._calculate_success_probability(current_sprint, simulation_results)
        
        # Métricas finais
        total_value = sum(self.backlog_df[self.backlog_df['issueid'].astype(str).isin(current_sprint)]['valor_trilha_a'])
        total_sp_estimado = sum(self.backlog_df[self.backlog_df['issueid'].astype(str).isin(current_sprint)]['storypoints'])
        
        expected_sp_efetivo = sum(
            np.mean(simulation_results['sp_efetivo_samples'][item_id])
            for item_id in current_sprint
        )
        
        metrics = {
            'prob_on_time': prob_on_time,
            'total_value': total_value,
            'total_sp_estimado': total_sp_estimado,
            'expected_sp_efetivo': expected_sp_efetivo,
            'n_items': len(current_sprint),
            'capacity_mean': self.capacity_distribution['mean']
        }
        
        return current_sprint, metrics
    
    def _greedy_selection_with_dependencies(self, candidates: List[Dict], 
                                           simulation_results: Dict) -> List[str]:
        """Seleção gulosa respeitando dependências"""
        selected = []
        G = self.dependency_graph.copy()
        
        while candidates:
            # Encontrar candidatos sem dependências não satisfeitas
            available = []
            for candidate in candidates:
                item_id = candidate['id']
                # Verificar se todas as dependências já estão selecionadas
                deps_satisfied = all(
                    dep in selected or dep not in G.nodes()
                    for dep in candidate['deps']
                )
                if deps_satisfied:
                    available.append(candidate)
            
            if not available:
                break
            
            # Escolher melhor disponível
            best_candidate = max(available, key=lambda x: x['marginal_gain'])
            item_id = best_candidate['id']
            
            # Verificar se cabe no sprint (usando percentil 80 da capacidade)
            capacity_p80 = np.percentile(simulation_results['capacity_samples'], 80)
            current_sp_efetivo = sum(
                np.percentile(simulation_results['sp_efetivo_samples'][sel_id], 80)
                for sel_id in selected
            )
            item_sp_efetivo = np.percentile(simulation_results['sp_efetivo_samples'][item_id], 80)
            
            if current_sp_efetivo + item_sp_efetivo <= capacity_p80:
                selected.append(item_id)
            
            # Remover candidato da lista
            candidates.remove(best_candidate)
        
        return selected
    
    def _calculate_success_probability(self, sprint_items: List[str], 
                                     simulation_results: Dict) -> float:
        """Calcula P(on-time) para um sprint"""
        if not sprint_items:
            return 1.0
        
        successes = 0
        for i in range(self.n_simulations):
            capacity = simulation_results['capacity_samples'][i]
            total_sp_efetivo = sum(
                simulation_results['sp_efetivo_samples'][item_id][i]
                for item_id in sprint_items
            )
            
            if total_sp_efetivo <= capacity:
                successes += 1
        
        return successes / self.n_simulations
    
    def _adjust_for_reliability(self, initial_sprint: List[str], 
                               simulation_results: Dict) -> List[str]:
        """Ajusta sprint para atingir meta de confiabilidade"""
        current_sprint = initial_sprint.copy()
        
        while (self._calculate_success_probability(current_sprint, simulation_results) < self.theta 
               and len(current_sprint) > 0):
            
            # Remover item com menor ganho marginal
            worst_item = None
            worst_gain = float('inf')
            
            for item_id in current_sprint:
                item_data = self.backlog_df[self.backlog_df['issueid'].astype(str) == item_id].iloc[0]
                expected_sp = np.mean(simulation_results['sp_efetivo_samples'][item_id])
                gain = item_data['valor_trilha_a'] / expected_sp
                
                if gain < worst_gain:
                    worst_gain = gain
                    worst_item = item_id
            
            if worst_item:
                current_sprint.remove(worst_item)
        
        return current_sprint
    
    def generate_execution_order(self, sprint_items: List[str]) -> List[str]:
        """Gera ordem de execução respeitando dependências"""
        if not sprint_items:
            return []
        
        # Subgrafo apenas com itens do sprint
        subgraph = self.dependency_graph.subgraph(sprint_items).copy()
        
        # Ordenação topológica com priorização por valor
        ordered = []
        while subgraph.nodes():
            # Encontrar nós sem dependências
            available = [node for node in subgraph.nodes() if subgraph.in_degree(node) == 0]
            
            if not available:
                # Se há ciclos, pegar qualquer nó
                available = list(subgraph.nodes())
            
            # Escolher por maior valor
            best_item = max(available, key=lambda x: 
                self.backlog_df[self.backlog_df['issueid'].astype(str) == x]['valor_trilha_a'].iloc[0]
            )
            
            ordered.append(best_item)
            subgraph.remove_node(best_item)
        
        return ordered
    
    def plan_sprint(self, 
                   trilha_b1_path: str = "/workspaces/codespaces-jupyter/data/Backlog - trilhaB1.csv",
                   trilha_b2_path: str = "/workspaces/codespaces-jupyter/data/Backlog - trilhaB2.csv",
                   trilha_b3_path: str = "/workspaces/codespaces-jupyter/data/Backlog - trilhaB3.csv", 
                   trilha_b4_path: str = "/workspaces/codespaces-jupyter/data/Backlog - trilhaB4.csv",
                   backlog_priorizado_path: str = "/workspaces/codespaces-jupyter/data/backlog_priorizado.csv") -> Dict:
        """Executa todo o pipeline de planejamento"""
        
        # Carregar dados
        self.load_and_process_data(
            trilha_b1_path, trilha_b2_path, trilha_b3_path,
            trilha_b4_path, backlog_priorizado_path
        )
        
        # Simulação Monte Carlo
        simulation_results = self.monte_carlo_simulation()
        
        # Otimização
        sprint_items, metrics = self.optimize_sprint(simulation_results)
        
        # Ordem de execução
        execution_order = self.generate_execution_order(sprint_items)
        
        # Compilar resultado final
        result = {
            'sprint_items': sprint_items,
            'execution_order': execution_order,
            'metrics': metrics,
            'items_details': self.backlog_df[
                self.backlog_df['issueid'].astype(str).isin(sprint_items)
            ][['issueid', 'valor_trilha_a', 'storypoints', 'deps_parsed']].to_dict('records')
        }
        
        return result
    
    def print_results(self, result: Dict) -> None:
        """Imprime resultados de forma organizada"""
        print("\n" + "="*60)
        print(f"📊 PLANEJAMENTO DE SPRINT - TEAM {self.team_id}")
        print("="*60)
        
        metrics = result['metrics']
        
        # Métricas principais
        print(f"\n🎯 MÉTRICAS PRINCIPAIS:")
        print(f"   Probabilidade de sucesso: {metrics['prob_on_time']:.1%} (meta: {self.theta:.1%})")
        print(f"   Valor total do sprint: {metrics['total_value']:.2f}")
        print(f"   Story Points estimados: {metrics['total_sp_estimado']}")
        print(f"   Story Points efetivos (esperado): {metrics['expected_sp_efetivo']:.1f}")
        print(f"   Capacidade média do time: {metrics['capacity_mean']:.1f} SP")
        print(f"   Número de itens: {metrics['n_items']}")
        
        # Indicadores de risco
        utilization = metrics['expected_sp_efetivo'] / metrics['capacity_mean']
        risk_level = "🟢 BAIXO" if utilization < 0.7 else "🟡 MÉDIO" if utilization < 0.9 else "🔴 ALTO"
        print(f"   Utilização da capacidade: {utilization:.1%} - {risk_level}")
        
        # Lista de itens selecionados
        print(f"\n📋 ITENS SELECIONADOS (Ordem de Execução):")
        for i, item_id in enumerate(result['execution_order'], 1):
            item_detail = next(
                (item for item in result['items_details'] if str(item['issueid']) == item_id), 
                None
            )
            if item_detail:
                deps_str = f" (deps: {item_detail['deps_parsed']})" if item_detail['deps_parsed'] else ""
                print(f"   {i:2d}. ID {item_id} | Valor: {item_detail['valor_trilha_a']:.2f} | SP: {item_detail['storypoints']}{deps_str}")
        
        # Recomendações
        print(f"\n💡 RECOMENDAÇÕES:")
        if metrics['prob_on_time'] < self.theta:
            print(f"   ⚠️  Probabilidade de sucesso abaixo da meta ({self.theta:.1%})")
            print(f"   📉 Considere remover itens ou revisar estimativas")
        else:
            print(f"   ✅ Sprint atende à meta de confiabilidade")
        
        if utilization > 0.85:
            print(f"   ⚠️  Alta utilização da capacidade - risco de spillover")
        
        # Itens de fronteira (não selecionados mas com alta prioridade)
        print(f"\n🔄 ITENS DE FRONTEIRA (próximos da seleção):")
        selected_ids = set(result['sprint_items'])
        frontier_items = self.backlog_df[
            ~self.backlog_df['issueid'].astype(str).isin(selected_ids)
        ].nlargest(3, 'valor_trilha_a')
        
        for _, item in frontier_items.iterrows():
            print(f"   • ID {item['issueid']} | Valor: {item['valor_trilha_a']:.2f} | SP: {item['storypoints']}")

    def save_results(self, result: Dict, output_path: str) -> None:
        """Salva resultados em CSV"""
        # Preparar dados para salvar
        output_data = []
        
        for i, item_id in enumerate(result['execution_order'], 1):
            item_detail = next(
                (item for item in result['items_details'] if str(item['issueid']) == item_id),
                None
            )
            
            if item_detail:
                output_data.append({
                    'execution_order': i,
                    'item_id': item_id,
                    'valor_trilha_a': item_detail['valor_trilha_a'],
                    'story_points': item_detail['storypoints'],
                    'dependencies': str(item_detail['deps_parsed']),
                    'has_dependencies': len(item_detail['deps_parsed']) > 0
                })
        
        # Adicionar métricas como linhas extras
        metrics = result['metrics']
        
        # Salvar
        df_output = pd.DataFrame(output_data)
        df_output.to_csv(output_path, index=False)
        
        # Salvar métricas em arquivo separado
        metrics_path = output_path.replace('.csv', '_metrics.txt')
        with open(metrics_path, 'w') as f:
            f.write(f"PLANEJAMENTO DE SPRINT - TEAM {self.team_id}\n")
            f.write("="*50 + "\n\n")
            f.write(f"Probabilidade de sucesso: {metrics['prob_on_time']:.1%}\n")
            f.write(f"Valor total: {metrics['total_value']:.2f}\n")
            f.write(f"SP estimados: {metrics['total_sp_estimado']}\n")
            f.write(f"SP efetivos esperados: {metrics['expected_sp_efetivo']:.1f}\n")
            f.write(f"Capacidade média: {metrics['capacity_mean']:.1f}\n")
            f.write(f"Número de itens: {metrics['n_items']}\n")
            f.write(f"Utilização: {metrics['expected_sp_efetivo']/metrics['capacity_mean']:.1%}\n")
        
        print(f"✅ Resultados salvos em: {output_path}")
        print(f"✅ Métricas salvas em: {metrics_path}")

# Função principal para execução
def main():
    """Exemplo de execução do planejamento de sprint"""
    
    # Parâmetros do planejador
    planner = SprintPlanner(
        team_id=1,
        theta=0.8,  # 80% de confiabilidade
        n_simulations=1000,  # Reduzido para exemplo rápido
        lambda_spillover=0.1,
        random_state=42
    )
    
    try:
        print("🚀 Iniciando planejamento de sprint...")
        
        # Executar planejamento completo
        result = planner.plan_sprint()
        
        # Mostrar resultados
        planner.print_results(result)
        
        # Salvar resultados
        planner.save_results(result, '/workspaces/codespaces-jupyter/data/sprint_planejado.csv')
        
        print("\n✅ Planejamento concluído com sucesso!")
        
        # Análise adicional
        print(f"\n🔍 ANÁLISE ADICIONAL:")
        
        # Verificar se há itens bloqueados
        blocked_items = planner.backlog_df[
            planner.backlog_df['deps_parsed'].apply(len) > 0
        ]
        if len(blocked_items) > 0:
            print(f"   📌 {len(blocked_items)} itens com dependências no backlog")
        
        # Distribuição de tamanhos
        size_dist = planner.backlog_df['storypoints'].value_counts().sort_index()
        print(f"   📊 Distribuição de tamanhos: {dict(size_dist)}")
        
        # Capacidade vs demanda
        total_backlog_sp = planner.backlog_df['storypoints'].sum()
        sprints_needed = total_backlog_sp / planner.capacity_distribution['mean']
        print(f"   ⏱️  Sprints estimados para todo o backlog: {sprints_needed:.1f}")
        
    except Exception as e:
        print(f"❌ Erro durante planejamento: {e}")
        import traceback
        traceback.print_exc()

# Classe utilitária para análise de sensibilidade
class SensitivityAnalyzer:
    """Análise de sensibilidade dos parâmetros do planejamento"""
    
    def __init__(self, planner: SprintPlanner):
        self.planner = planner
    
    def analyze_theta_sensitivity(self, theta_range: List[float]) -> pd.DataFrame:
        """Analisa como diferentes valores de theta afetam o sprint"""
        results = []
        
        original_theta = self.planner.theta
        simulation_results = self.planner.monte_carlo_simulation()
        
        for theta in theta_range:
            self.planner.theta = theta
            sprint_items, metrics = self.planner.optimize_sprint(simulation_results)
            
            results.append({
                'theta': theta,
                'n_items': metrics['n_items'],
                'total_value': metrics['total_value'],
                'prob_on_time': metrics['prob_on_time'],
                'sp_estimado': metrics['total_sp_estimado'],
                'sp_efetivo': metrics['expected_sp_efetivo']
            })
        
        # Restaurar theta original
        self.planner.theta = original_theta
        
        return pd.DataFrame(results)
    
    def analyze_capacity_impact(self, capacity_adjustments: List[float]) -> pd.DataFrame:
        """Analisa impacto de diferentes cenários de capacidade"""
        results = []
        
        original_capacity = self.planner.capacity_distribution.copy()
        
        for adjustment in capacity_adjustments:
            # Ajustar capacidade
            self.planner.capacity_distribution['mean'] = original_capacity['mean'] * adjustment
            
            # Re-executar simulação e otimização
            simulation_results = self.planner.monte_carlo_simulation()
            sprint_items, metrics = self.planner.optimize_sprint(simulation_results)
            
            results.append({
                'capacity_factor': adjustment,
                'capacity_mean': self.planner.capacity_distribution['mean'],
                'n_items': metrics['n_items'],
                'total_value': metrics['total_value'],
                'prob_on_time': metrics['prob_on_time']
            })
        
        # Restaurar capacidade original
        self.planner.capacity_distribution = original_capacity
        
        return pd.DataFrame(results)

if __name__ == "__main__":
    main()

🚀 Iniciando planejamento de sprint...
1. Carregando arquivos...
2. Preparando dados...
   Backlog preparado: 28 itens
   Ciclo quebrado: removida aresta ('22', '21')
   Grafo: 28 nós, 31 arestas
3. Treinando modelos preditivos...
   Capacidade: μ=9.2, σ=1.9
   Modelo overrun treinado (MAE: 0.253)
   Spillover rates: {'S': 0.5, 'M': 0.5, 'L': 0.5, 'XL': 0.5}
✅ Dados carregados e modelos treinados!
4. Executando 1000 simulações Monte Carlo...
5. Otimizando seleção de itens...

📊 PLANEJAMENTO DE SPRINT - TEAM 1

🎯 MÉTRICAS PRINCIPAIS:
   Probabilidade de sucesso: 97.1% (meta: 80.0%)
   Valor total do sprint: 9.00
   Story Points estimados: 6
   Story Points efetivos (esperado): 5.5
   Capacidade média do time: 9.2 SP
   Número de itens: 4
   Utilização da capacidade: 59.5% - 🟢 BAIXO

📋 ITENS SELECIONADOS (Ordem de Execução):
    1. ID 13 | Valor: 2.25 | SP: 3
    2. ID 1 | Valor: 2.25 | SP: 1
    3. ID 2 | Valor: 2.25 | SP: 1 (deps: ['1'])
    4. ID 3 | Valor: 2.25 | SP: 1 (deps: ['1'])

